# Using Table and Data 

### Importing Libraries

In [1]:
import sqlite3


In [2]:
# create connection  and cursor object
connection = sqlite3.connect("dannys_dinner_database")
cursor = connection.cursor()

In [3]:
# see the tables in database
query ="select name from sqlite_master  where type='table';"
cursor.execute(query)
result = cursor.fetchall()
print('Tables: {}'.format(result))

Tables: [('sales',), ('menu',), ('members',)]


### Question and Answer

What is the total amount each customer spent at the restaurant?

In [4]:
query= """
select s.customer_id,sum(price) 
from sales s inner join menu m 
on s.product_id = m.product_id
group by s.customer_id 
"""
cursor.execute(query)
result = cursor.fetchall()
for record in result:
    print(record)

('A', 76)
('B', 74)
('C', 36)


How many days has each customer visited the restaurant?

In [13]:


query= """
select customer_id,count(distinct(order_date))
from sales
group by customer_id;

"""
cursor.execute(query)
result = cursor.fetchall()
for record in result:
    print(record)

('A', 4)
('B', 6)
('C', 2)


What was the first item from the menu purchased by each customer?

In [21]:


query= """
select customer_id,product_name,s.order_date
from sales s inner join menu m
on s.product_id = m.product_id
group by s.customer_id
having s.order_date = min(s.order_date)



"""
cursor.execute(query)
result = cursor.fetchall()
for record in result:
    print(record)

('A', 'sushi', '2021-01-01')
('B', 'curry', '2021-01-01')
('C', 'ramen', '2021-01-01')


What is the most purchased item on the menu and how many times was it purchased by all customers?

In [23]:


query= """
select m.product_name, count(*) as item_count
from sales s inner join menu m 
    on s.product_id = m.product_id 
group by s.product_id
order by item_count desc
limit 1;


"""
cursor.execute(query)
result = cursor.fetchall()
for record in result:
    print(record)

('ramen', 8)


Which item was the most popular for each customer?

In [36]:


query= """
select s.customer_id,m.product_name, count(*) as purchase_count
from sales s inner join menu m 
    on s.product_id = m.product_id 
group by s.customer_id,s.product_id
having purchase_count= (
select max(purchase_count)
from (
select s.customer_id,m.product_name, count(*) as purchase_count
from sales s inner join menu m 
    on s.product_id = m.product_id 
group by s.customer_id,s.product_id
) as subquery
where subquery.customer_id = s.customer_id
)

"""
cursor.execute(query)
result = cursor.fetchall()
for record in result:
    print(record)

('A', 'ramen', 3)
('B', 'sushi', 2)
('B', 'curry', 2)
('B', 'ramen', 2)
('C', 'ramen', 3)


Which item was purchased first by the customer after they became a member?

In [57]:
query1="""select s.customer_id,min(s.order_date)
    from sales s join members me on 
    s.customer_id = me.customer_id
    where s.order_date>me.join_date
    group by s.customer_id
"""

query= """
    select s.customer_id, m.product_name
    from sales s join menu m on
    s.product_id= m.product_id join members me on
    s.customer_id= me.customer_id
    where (s.customer_id,s.order_date) in (select s.customer_id,min(s.order_date)
    from sales s join members me on 
    s.customer_id = me.customer_id
    where s.order_date>me.join_date
    group by s.customer_id)
    
    

"""
cursor.execute(query)
result = cursor.fetchall()
for record in result:
    print(record)

('A', 'ramen')
('B', 'sushi')


Which item was purchased just before the customer became a member?

In [82]:


query= """
select s.customer_id, m.product_name
    from sales s join menu m on
    s.product_id= m.product_id join members me on
    s.customer_id= me.customer_id
where (s.customer_id,s.order_date) in (select s.customer_id,max(s.order_date) as orderdate
    from sales s join members me on 
    s.customer_id = me.customer_id
    where s.order_date<me.join_date
    group by s.customer_id )

"""
cursor.execute(query)
result = cursor.fetchall()
for record in result:
    print(record)

('A', 'sushi')
('A', 'curry')
('B', 'sushi')


What is the total items and amount spent for each member before they became a member?

In [84]:
query= """
select s.customer_id,count(s.product_id) as total_items, sum(m.price)
from sales s join menu m on
    s.product_id= m.product_id join members me on
    s.customer_id= me.customer_id
where s.order_date<me.join_date
group by s.customer_id 

"""
cursor.execute(query)
result = cursor.fetchall()
for record in result:
    print(record)

('A', 2, 25)
('B', 3, 40)


If each $1 spent equates to 10 points and sushi has a 2x points multiplier - how many points would each customer have?

In [88]:
query= """
select s.customer_id,
count(s.product_id) as total_items, 
sum(case when m.product_name = 'sushi' then 20 * m.price else 10*m.price end) as total_points
from sales s join menu m on
    s.product_id= m.product_id 

group by s.customer_id 

"""
cursor.execute(query)
result = cursor.fetchall()
for record in result:
    print(record)

('A', 6, 860)
('B', 6, 940)
('C', 3, 360)


In the first week after a customer joins the program (including their join date) they earn 2x points on all items, not just sushi - how many points do customer A and B have at the end of January?

In [105]:
query= """
select s.customer_id,
count(s.product_id) as total_items, 
sum(case 
    when (julianday(s.order_date)-julianday(me.join_date)) between 0 and 7 then 20 * m.price
    when m.product_name = 'sushi' then 20 * m.price 
    else 10*m.price 
    end)as total_points
    
from sales s join menu m on
    s.product_id= m.product_id
        join members me on s.customer_id = me.customer_id

group by s.customer_id 

"""
cursor.execute(query)
result = cursor.fetchall()
for record in result:
    print(record)

('A', 6, 1370)
('B', 6, 1060)


The following questions are related creating basic data tables that Danny and his team can use to quickly derive insights without needing to join the underlying tables using SQL.

Recreate the following table output using the available data:

<br>
<img src="images/table1.png" alt="table 1" width="500">

In [9]:
query= """
select s.customer_id,s.order_date,m.product_name,m.price,
case
    when me.join_date is null then 'N'
    when s.order_date < me.join_date then "N"
    else 'Y'
end as member

 from sales s full outer join menu m on
    s.product_id= m.product_id full outer join members me on
    s.customer_id= me.customer_id

"""
cursor.execute(query)
result = cursor.fetchall()
for record in result:
    print(record)

('A', '2021-01-01', 'sushi', 10, 'N')
('A', '2021-01-01', 'curry', 15, 'N')
('A', '2021-01-07', 'curry', 15, 'Y')
('A', '2021-01-10', 'ramen', 12, 'Y')
('A', '2021-01-11', 'ramen', 12, 'Y')
('A', '2021-01-11', 'ramen', 12, 'Y')
('B', '2021-01-01', 'curry', 15, 'N')
('B', '2021-01-02', 'curry', 15, 'N')
('B', '2021-01-04', 'sushi', 10, 'N')
('B', '2021-01-11', 'sushi', 10, 'Y')
('B', '2021-01-16', 'ramen', 12, 'Y')
('B', '2021-02-01', 'ramen', 12, 'Y')
('C', '2021-01-01', 'ramen', 12, 'N')
('C', '2021-01-01', 'ramen', 12, 'N')
('C', '2021-01-07', 'ramen', 12, 'N')


Danny also requires further information about the ranking of customer products, but he purposely does not need the ranking for non-member purchases so he expects null ranking values for the records when customers are not yet part of the loyalty program.

<br>
<img src="images/table2.png" alt="table 2" width="500">

In [10]:
query= """
select z.*
,case when z.member='N' then null 
else rank() over(partition by z.customer_id order by z.price desc)
end as ranking

from 
(
select s.customer_id,s.order_date,m.product_name,m.price,
case
    when me.join_date is null then 'N'
    when s.order_date < me.join_date then "N"
    else 'Y'
end as member
from sales s join menu m on
    s.product_id= m.product_id Left outer join members me on
    s.customer_id= me.customer_id
) z
order by z.customer_id,z.member

"""
cursor.execute(query)
result = cursor.fetchall()
for record in result:
    print(record)

('A', '2021-01-01', 'curry', 15, 'N', None)
('A', '2021-01-01', 'sushi', 10, 'N', None)
('A', '2021-01-07', 'curry', 15, 'Y', 1)
('A', '2021-01-10', 'ramen', 12, 'Y', 3)
('A', '2021-01-11', 'ramen', 12, 'Y', 3)
('A', '2021-01-11', 'ramen', 12, 'Y', 3)
('B', '2021-01-01', 'curry', 15, 'N', None)
('B', '2021-01-02', 'curry', 15, 'N', None)
('B', '2021-01-04', 'sushi', 10, 'N', None)
('B', '2021-01-16', 'ramen', 12, 'Y', 3)
('B', '2021-02-01', 'ramen', 12, 'Y', 3)
('B', '2021-01-11', 'sushi', 10, 'Y', 5)
('C', '2021-01-01', 'ramen', 12, 'N', None)
('C', '2021-01-01', 'ramen', 12, 'N', None)
('C', '2021-01-07', 'ramen', 12, 'N', None)


In [11]:
# include it in finally in try-except statement.
if connection:
    connection.close()